In [1]:
from pathlib import Path 
import os, dotenv, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

dotenv.load_dotenv()
os.chdir(Path(config["pythonpath"]).expanduser())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from src.preprocessing.quality_process import compute_file_metrics

In [3]:
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer

VERSION = config["version"]
VARIATION = "nocomments"

DATA1 = f"data/good_syntax/source_{VARIATION}_{VERSION}.jsonl"
DATA2 = f"data/good_syntax/augmented_verified_{VERSION}.jsonl"
MODEL_ID = config["model_id"]

codellama_tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=False,
)

if codellama_tokenizer.pad_token is None:
    codellama_tokenizer.pad_token = codellama_tokenizer.eos_token

# from quality_process import compute_file_metrics  # <-- uncomment and fix path

results = []
with open(DATA1, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Scoring rows"):
        row = json.loads(line)
        results.append(
            compute_file_metrics(
                row["filename"],
                row["content"],
                model_tokenizer=codellama_tokenizer,
            )
        )
with open(DATA2, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Scoring rows"):
        row = json.loads(line)
        results.append(
            compute_file_metrics(
                row["filename"],
                row["content"],
                model_tokenizer=codellama_tokenizer,
            )
        )

df = pd.DataFrame(results)


/opt/anaconda3/envs/Automated_Reasoning_for_Cryptography/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Scoring rows: 326it [00:00, 1928.42it/s]
Scoring rows: 916it [00:00, 5134.24it/s]


In [4]:
df.head()

,filename,sha1,bytes,lines,avg_line_len,max_line_len,non_ascii_ratio,binary_like,enc_total_matched,enc_max_run,enc_fraction,enc_hits_base64,enc_hits_hexbytes,enc_hits_unicode,num_tokens_lang,k_shingle,num_shingles,hexnum_ratio,num_tokens_model,junk_path
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,6502af5ae03bf06118116731814e2701b6ceefd6,518,19,26.32,75,0.0,False,0,0,0.0,0,0,0,138,5,134,0.0,173,False
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,524b0175819d4abbc5cfe239a06cf31f633a177a,651,28,22.29,47,0.0,False,0,0,0.0,0,0,0,124,5,120,0.0,214,False
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,f13fdcd8487b90dd5282df8321131ef2693b39bc,649,28,22.21,47,0.0,False,0,0,0.0,0,0,0,124,5,120,0.0,214,False
3,AES-GCM-SIV-proof/proof/asm/cryptol/X86.cry,fccd1c63ef1fe602a316468b1108a3a131afd897,1263,47,25.89,62,0.0,False,0,0,0.0,0,0,0,359,5,355,0.0,465,False
4,aws-lc-verification/cryptol-specs/Common/Field...,327f8f5aa300e2c58573eee9c8b780c7980087bf,375,22,16.09,52,0.0,False,0,0,0.0,0,0,0,105,5,101,0.0,138,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   filename           1242 non-null   object 
 1   sha1               1242 non-null   object 
 2   bytes              1242 non-null   int64  
 3   lines              1242 non-null   int64  
 4   avg_line_len       1242 non-null   float64
 5   max_line_len       1242 non-null   int64  
 6   non_ascii_ratio    1242 non-null   float64
 7   binary_like        1242 non-null   bool   
 8   enc_total_matched  1242 non-null   int64  
 9   enc_max_run        1242 non-null   int64  
 10  enc_fraction       1242 non-null   float64
 11  enc_hits_base64    1242 non-null   int64  
 12  enc_hits_hexbytes  1242 non-null   int64  
 13  enc_hits_unicode   1242 non-null   int64  
 14  num_tokens_lang    1242 non-null   int64  
 15  k_shingle          1242 non-null   int64  
 16  num_shingles       1242 

In [6]:
# StarCoder-like thresholds (tune if needed)
MAX_BYTES         = 200_000
MAX_NONASCII      = 0.20
ENC_MAX_RUN_CHARS = 1024
ENC_MAX_FRACTION  = 0.50
MAX_LINES_TOTAL   = 100_000
MAX_LINE_AVG_LEN  = 100
MAX_LINE_MAX_LEN  = 1_000
MIN_TOKENS_LANG   = 40      # language-token gate (Cryptol tokenizer)
MAX_TOKENS_LANG   = 10_000  # optional upper bound
MIN_TOKENS_MODEL  = 40      # only if you’ve populated num_tokens_model
MAX_TOKENS_MODEL  = 1600
MAX_HEXNUM_RATIO  = 0.20


# --- exact dedup (keep first occurrence of each sha1) ---
# mark duplicates (True means "is duplicate" => drop later)
dup_mask = df.duplicated(subset=["sha1"], keep="first")

# --- encoded data (StarCoder) ---
enc_mask = (df["enc_max_run"] > ENC_MAX_RUN_CHARS) | (df["enc_fraction"] > ENC_MAX_FRACTION)

# --- long-line filters (StarCoder) ---
longline_mask = (
    (df["lines"] > MAX_LINES_TOTAL) |
    (df["avg_line_len"] > MAX_LINE_AVG_LEN) |
    (df["max_line_len"] > MAX_LINE_MAX_LEN)
)

# --- binary-like content ---
binary_mask = df["binary_like"].fillna(False)

# --- non-ascii density ---
nonascii_mask = df["non_ascii_ratio"].fillna(0) > MAX_NONASCII

# --- size guardrail (bytes) ---
bytes_mask = df["bytes"].fillna(0) > MAX_BYTES

# --- language-token bounds ---
lang_small_mask = df["num_tokens_model"].fillna(0) < MIN_TOKENS_MODEL
lang_large_mask = df["num_tokens_model"].fillna(0) > MAX_TOKENS_MODEL

# --- shingles exist (needed for Jaccard) ---
no_shingles_mask = df["num_shingles"].fillna(0) <= 0

# --- numeric/hex blob concentration ---
hexnum_mask = df["hexnum_ratio"].fillna(0) > MAX_HEXNUM_RATIO

# --- model-token gate (only apply where available) ---
if "num_tokens_model" in df.columns:
    model_small_mask = df["num_tokens_model"].fillna(np.inf) < MIN_TOKENS_MODEL
else:
    model_small_mask = pd.Series(False, index=df.index)


In [7]:
# Combine all hard-drop reasons
drop_mask = (
    dup_mask |
    enc_mask |
    longline_mask |
    binary_mask |
    nonascii_mask |
    bytes_mask |
    lang_small_mask |
    lang_large_mask |
    no_shingles_mask |
    hexnum_mask |
    model_small_mask
)

# Optional: compute a human-readable fail reason (first rule that tripped)
def first_reason(i):
    if dup_mask.iat[i]:          return "exact_duplicate"
    if enc_mask.iat[i]:          return "encoded_data"
    if longline_mask.iat[i]:     return "long_lines"
    if binary_mask.iat[i]:       return "binary_like"
    if nonascii_mask.iat[i]:     return "too_much_nonascii"
    if bytes_mask.iat[i]:        return "too_large_bytes"
    if lang_small_mask.iat[i]:   return "too_few_lang_tokens"
    if lang_large_mask.iat[i]:   return "too_many_lang_tokens"
    if no_shingles_mask.iat[i]:  return "no_shingles"
    if hexnum_mask.iat[i]:       return "hexnum_blob"
    if model_small_mask.iat[i]:  return "too_few_model_tokens"
    return "ok"

df = df.copy()
df["quality_ok"] = ~drop_mask
df["fail_reason"] = [first_reason(i) for i in range(len(df))]


In [8]:
dedup_cols = [
    "filename", "sha1",
    # size/lines
    "bytes", "lines", "avg_line_len", "max_line_len",
    # content/encoding
    "non_ascii_ratio", "binary_like",
    "enc_total_matched", "enc_max_run", "enc_fraction",
    "enc_hits_base64", "enc_hits_hexbytes", "enc_hits_unicode",
    # tokens/shingles
    "num_tokens_lang", "k_shingle", "num_shingles", "hexnum_ratio",
    # model tokens (optional)
    "num_tokens_model",
    # path heuristic & status
    "quality_ok", "fail_reason",
]

candidate_df = df.loc[df["quality_ok"], dedup_cols].reset_index(drop=True)

temp1 = pd.read_json(DATA1, lines=True)
temp2 = pd.read_json(DATA2, lines=True)

similar_process_df = pd.concat([temp1, temp2], ignore_index=True)
similar_process_df = similar_process_df[similar_process_df['filename'].isin(candidate_df['filename'])].reset_index(drop=True)

put_back_path = Path("data/dropped/files_to_put_back.csv")
'''
if put_back_path.exists():
    put_back_set = pd.read_csv(put_back_path)
    put_back_filenames = set(put_back_set["filename"].dropna().tolist())
else:
    put_back_filenames = set()

for fname in put_back_filenames:
    if fname in df['filename'].values:
        candidate_df = pd.concat([candidate_df, df[df['filename'] == fname][dedup_cols]], ignore_index=True)
        candidate_df.loc[candidate_df['filename'] == fname, 'quality_ok'] = True
        '''
similar_process_df.head()

,filename,filetype,content,variant,n_imports_original,n_imports_final
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cry,module AES where\n\nimport `Common::AES\n\ntyp...,without_comments,NaN,NaN
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cry,module AES128 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cry,module AES256 where\n\nimport `Common::AES\nim...,without_comments,NaN,NaN
3,AES-GCM-SIV-proof/proof/asm/cryptol/X86.cry,cry,module X86 where\n\nimport AES128\n\naesenc : ...,without_comments,NaN,NaN
4,aws-lc-verification/cryptol-specs/Common/Field...,cry,module Common::Field where\n\nimport Common::M...,without_comments,NaN,NaN


In [9]:
print("[summary] total:", len(df))
print("[summary] kept :", int(df["quality_ok"].sum()))
print("[summary] dropped:", int((~df["quality_ok"]).sum()))
print("[summary] drop reasons:")
print(df.loc[~df["quality_ok"], "fail_reason"].value_counts())


[summary] total: 1242
[summary] kept : 969
[summary] dropped: 273
[summary] drop reasons:
fail_reason
too_few_lang_tokens    206
exact_duplicate         47
hexnum_blob             10
encoded_data             7
no_shingles              3
Name: count, dtype: int64


In [10]:
dropped = df[df["quality_ok"] == False].copy().reset_index(drop=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   filename           1242 non-null   object 
 1   sha1               1242 non-null   object 
 2   bytes              1242 non-null   int64  
 3   lines              1242 non-null   int64  
 4   avg_line_len       1242 non-null   float64
 5   max_line_len       1242 non-null   int64  
 6   non_ascii_ratio    1242 non-null   float64
 7   binary_like        1242 non-null   bool   
 8   enc_total_matched  1242 non-null   int64  
 9   enc_max_run        1242 non-null   int64  
 10  enc_fraction       1242 non-null   float64
 11  enc_hits_base64    1242 non-null   int64  
 12  enc_hits_hexbytes  1242 non-null   int64  
 13  enc_hits_unicode   1242 non-null   int64  
 14  num_tokens_lang    1242 non-null   int64  
 15  k_shingle          1242 non-null   int64  
 16  num_shingles       1242 

In [11]:
#review_data_set = df[~df['fail_reason'].isin(['ok', 'exact_duplicate'])].copy().reset_index(drop=True)
#out_path = Path(f"data/dropped/review_files_{VARIATION}_{VERSION}.csv")
#out_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing
#review_data_set.to_csv(out_path, index=False)


In [12]:
from src.preprocessing.similiar_process import run_from_dataframe

# candidate_df must have an absolute-path 'filename' column.
df_files, df_pairs, similar_files = run_from_dataframe(
    similar_process_df,
    filename_col="filename",
    content_col="content",
    out_dir="minhash_outputs",
)


[info] ==== Starting MinHash/LSH over DataFrame ====
[info] params: K_SHINGLE=5, NUM_PERM=512, LSH_THRESHOLD=0.7
[info] loaded 969 files from candidate_df
[info] files indexed   : 969
[diag] total candidate pairs: 350
[diag] pairs with jaccard >= 0.7: 289
[info] wrote CSV and Parquet to minhash_outputs/

[info] ==== MinHash/LSH run summary ====
[info] files loaded  : 969
[info] files indexed : 969
[info] files with 0 shingles (tokens < 5): 0
[info] candidate pairs (from LSH) : 350
[info] pairs with Jaccard >= 0.60: 341
[info] pairs with Jaccard >= 0.70: 289
[info] pairs with Jaccard >= 0.80: 194
[info] pairs with Jaccard >= 0.85: 142
[info] pairs with Jaccard >= 0.90: 101
[info] avg Jaccard (candidates)  : 0.8139
[info] max Jaccard               : 1.0000
[info] min Jaccard               : 0.4922

[info] top pairs:
                                                                                        a                                                                                     

In [13]:
df_files.head()

,filename,num_tokens,num_shingles,num_perm,k_shingle,minhash_hashvalues
0,AES-GCM-SIV-proof/proof/asm/cryptol/X86.cry,359,259,512,5,"[1884889, 40571396, 3229112, 373781, 775270, 3..."
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,138,107,512,5,"[63534513, 98619623, 8847283, 54209211, 161726..."
2,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,124,117,512,5,"[111483891, 50663666, 30189362, 14879480, 4002..."
3,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,124,117,512,5,"[7807519, 8228268, 14633020, 6262864, 26960892..."
4,BLST-Verification/spec/BLSFieldExtras.cry,623,544,512,5,"[1562758, 22833707, 3178118, 2336108, 18354179..."


In [14]:
from src.preprocessing.cluster_process import run_clustering

# If you already have df_files/df_pairs in memory:
df_keep, df_drop, df_clusters = run_clustering(
    df_files=df_files,          # from similiar_process
    df_pairs=df_pairs,          # from similiar_process
    jaccard_keep_threshold=0.70,
    out_dir="minhash_outputs",
    content_lookup=None,        # or {filename: raw_text} if you want text-derived penalties
    save_outputs=True
)

[info] clusters formed   : 803
[info] kept files        : 803
[info] dropped files     : 166
[info] wrote keep/drop/cluster CSVs to minhash_outputs/


In [15]:
dataset = df[df['filename'].isin(df_keep['filename'].tolist())].copy()
df.loc[
    df['filename'].isin(df_drop['filename'].tolist()),
    'quality_ok'
    ] = False
df.loc[
    df['filename'].isin(df_drop['filename'].tolist()),
    'fail_reason'
    ] = 'similiar_file_exists'

dropped = df[~df['filename'].isin(dataset['filename'].tolist())].copy().reset_index(drop=True)


In [16]:
#dropped.to_csv(f"data/dropped/{VARIATION}_dropped_files_{VERSION}.csv", index=False)

In [17]:
import re

verified_files = set(dataset["filename"].unique())

#all_files_df = pd.read_json(DATA1, lines=True)
all_files_filtered_df = similar_process_df[similar_process_df["filename"].isin(verified_files)].reset_index(drop=True)

pattern = r"/\d{3}_testCase"   # slash + exactly 3 digits + "_testCase"
mask = all_files_filtered_df["filename"].astype(str).str.contains(pattern, regex=True, na=False)

test_case_df = all_files_filtered_df[mask].copy().reset_index(drop=True)
all_files_filtered_df = all_files_filtered_df[~mask].reset_index(drop=True)

mask = all_files_filtered_df["filename"].astype(str).str.contains(r"/tests/", regex=True, na=False)
all_files_filtered_df = all_files_filtered_df[~mask]

out_path = Path(f"data/clean_datasets/{VARIATION}_{VERSION}.jsonl")
out_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing
all_files_filtered_df.to_json(out_path, lines=True, orient="records")
test_case_df.to_json(f"data/clean_datasets/testCases_{VARIATION}_{VERSION}.jsonl", lines=True, orient="records")

In [18]:
df.to_csv(f"data/{VARIATION}_file_metrics_{VERSION}.csv", index=False)